In [7]:
!pip install q torch peft==0.4.0 transformers==4.31.0 trl==0.4.7 accelerate einops tqdm scipy
!pip install bitsandbytes-cuda110 bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 27.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 95.2 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

In [10]:
import os
from dataclasses import dataclass, field
from typing import Optional
import torch
from datasets import load_dataset, load_from_disk
from peft import LoraConfig, prepare_model_for_kbit_training
from transformers import (
AutoModelForCausalLM,
AutoTokenizer,
BitsAndBytesConfig,
HfArgumentParser,
TrainingArguments
) #Text generation task
from tqdm.notebook import tqdm #progress bard
from trl import SFTTrainer #SFT Training
from huggingface_hub import interpreter_login
import pandas as pd
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .


Token:  ········
Add token as git credential? (Y/n)  n


Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from huggingface_hub import create_repo
create_repo("shaunck96/mhGPT")

In [11]:
#Convert to a csv
#Loading Mental Health DataSet - load dataset with question answers about mental health topics
dataset = load_dataset("ani24/linkedinjobprompt", split='train')
df = pd.DataFrame(dataset)
df.head(4)

Generating train split: 0 examples [00:00, ? examples/s]

,prompt,output
0,"For the following details, generate a Linkedin...",Job Title: Senior Data Engineer Location: Alex...
1,"For the following details, generate a Linkedin...","Ethicon, part of Johnson & Johnson Medical Dev..."
2,"For the following details, generate a Linkedin...",Senior Software Engineer The mission of Micros...
3,"For the following details, generate a Linkedin...",Senior Software Engineer The mission of Micros...


In [19]:
df = df[~df['output'].isna()]

In [21]:
def format_row(row):
    question = row['prompt']
    answer = row['output']
    formatted_string = f"[INST] {question} [/INST] {answer} " #format used for llama
    return formatted_string

df['Formatted'] = df.apply(format_row, axis=1)
new_df = df.rename(columns = {'Formatted':'Text'})
new_df = new_df[['Text']]
new_df['Text'].iloc[0]

"[INST] For the following details, generate a Linkedin Job description \nCompany Name:CyberCoders\nJob Title:Senior Data Engineer\nEmployment Type: Full-time\nIndustry: Broadcast Media\nJob Function: Information Technology\nSeniority Level: Mid-Senior level\nExperience in years: 0.0\nEducation Level: bachelor degree\nLocation: Alexandria, VA [/INST] Job Title: Senior Data Engineer Location: Alexandria, VA Salary Range: $120k - $150k Requirements: ETL/ELT, SQL, AWS/Google Cloud, Linux/Unix, Spark (preferred), NoSQL (preferred), Machine Learning concepts (preferred) Based in beautiful Alexandria, VA, we are one of the hottest media analytics and software start-ups in the DC area. Due to growth, we are actively seeking to hire a Senior Data Engineer to join our team. The ideal candidate will have at least 5 years of experience with data pipelines (built with Python), ETL experience, a strong SQL background, experience with AWS or Google Cloud, and strong Unix/Linux fundamentals. Any exper

In [22]:
new_df.to_csv("formatted_data.csv", index=False)

In [23]:
training_dataset = load_dataset("csv", data_files="formatted_data.csv", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [24]:
#write login
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .


Token:  ········
Add token as git credential? (Y/n)  n


Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [25]:
from huggingface_hub import create_repo
create_repo("shaunck96/job_desc_gen")

RepoUrl('https://huggingface.co/shaunck96/job_desc_gen', endpoint='https://huggingface.co', repo_type='model', repo_id='shaunck96/job_desc_gen')

In [ ]:
#Fine Tuning
base_model = "meta-llama/Llama-2-7b-chat-hf"
new_model = "llama2-job-description-inferencer"
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

#QLora quantization Config (4-bit)
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4", #default quanitzation makes it slower ; nf4 is a special datatype to speed up calculation when input sequence is too long ; helpts to seleect split marker faster than traiditonal qlora
    bnb_4bit_compute_dypte = torch.float16,
    bnb_4bit_use_double_quant = False
)

#phi2 doesn't support flash attention at the time of writing this notebook
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config = bnb_config,
    trust_remote_code = True,
    flash_attn = True,
    flash_rotary = True,
    fused_dense = True,
    low_cpu_mem_usage = True,
    device_map = 0#,
    #revision = "refs/pr/23"
)

model.config.use_cache = False
model.config.pretraining_tp = 1

model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

training_arguments = TrainingArguments(
    output_dir = "./job_desc_gen",
    num_train_epochs = 2,
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 32,
    evaluation_strategy = "steps",
    eval_steps = 1500,
    logging_steps = 15,
    optim = "paged_adamw_8bit",
    learning_rate = 2e-4,
    save_steps = 1500,
    warmup_ratio = 0.05,
    weight_decay = 0.01,
    max_steps = -1
)

peft_config = LoraConfig(
    r = 32,
    lora_alpha = 64,
    lora_dropout = 0.05,
    bias = "none",
    task_type = "CAUSAL_LM",
    target_modules = ["Wqkv", "fc1", "fc2"]
)

trainer = SFTTrainer(
    model = model,
    train_dataset = training_dataset,
    peft_config = peft_config,
    dataset_text_field = "Text",
    max_seq_length = 690,
    tokenizer = tokenizer,
    args = training_arguments
)

#35-40 mins for 2 epochs
trainer.train()
trainer.save_model()

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]